In [ ]:
import sys
!{sys.executable} -m pip install opencv-python

In [3]:
import cv2
import numpy as np
import csv
import os
from tqdm.notebook import tqdm

def feature_tracking(prev_img, curr_img, prev_points):
    lk_params = dict(winSize=(21, 21), maxLevel=3,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    curr_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_img, curr_img, prev_points, None, **lk_params)
    
    if curr_points is None or status is None:
        return None, None
    
    good_old = prev_points[status.flatten() == 1]
    good_new = curr_points[status.flatten() == 1]
    
    return good_old, good_new

def detect_features(image, mask):
    feature_params = dict(maxCorners=200, qualityLevel=0.01, minDistance=30, blockSize=7)
    points = cv2.goodFeaturesToTrack(image, mask=mask, **feature_params)
    return points

def process_frames(dataset_path, mask_path=None, csv_file_path='results.csv'):
    frames_path = os.path.join(dataset_path, 'mav0', 'cam0', 'data')
    if not os.path.exists(frames_path):
        print(f"Frames path {frames_path} does not exist. Skipping this dataset.")
        return
    
    frames = sorted([os.path.join(frames_path, f) for f in os.listdir(frames_path) if f.endswith('.png')])
    
    if not frames:
        print(f"No frames found in {frames_path}")
        return
    
    prev_frame = cv2.imread(frames[0])
    if prev_frame is None:
        print("Cannot read the first frame")
        return
    
    frame_height, frame_width = prev_frame.shape[:2]
    
    mask = None
    if mask_path and os.path.exists(mask_path):
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None or mask.shape != (frame_height, frame_width):
            print("Mask image not found or dimensions do not match the video frame. No mask will be used.")
            mask = None
        else:
            print(f"Mask loaded from {mask_path}")
    
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_points = detect_features(prev_frame_gray, mask)
    
    if prev_points is None:
        print("No initial features detected. Exiting.")
        return
    
    trajectory = np.zeros((3, 1))
    poses = [trajectory]
    
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['frame_num', 'x', 'y', 'z'])
        
        frame_num = 0
        
        with tqdm(total=len(frames), desc=f"Processing {os.path.basename(dataset_path)}") as pbar:
            for frame_path in frames[1:]:
                frame = cv2.imread(frame_path)
                if frame is None:
                    continue
                
                curr_frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                prev_points, curr_points = feature_tracking(prev_frame_gray, curr_frame_gray, prev_points)
                
                if prev_points is None or curr_points is None or len(prev_points) < 5 or len(curr_points) < 5:
                    print("Not enough points to compute the essential matrix, re-detecting features")
                    prev_points = detect_features(prev_frame_gray, mask)
                    curr_points = detect_features(curr_frame_gray, mask)
                    if prev_points is None or curr_points is None or len(prev_points) < 5 or len(curr_points) < 5:
                        print("Not enough points detected, repeating last trajectory")
                        x, y, z = trajectory.flatten()
                        writer.writerow([frame_num, x, y, z])
                        prev_frame_gray = curr_frame_gray.copy()
                        frame_num += 1
                        pbar.update(1)
                        continue
                
                try:
                    prev_points = prev_points.reshape(-1, 2)
                    curr_points = curr_points.reshape(-1, 2)
                    prev_points = prev_points.astype(np.float32)
                    curr_points = curr_points.astype(np.float32)
                    
                    E, mask_e = cv2.findEssentialMat(curr_points, prev_points, focal=1.0, pp=(0.0, 0.0), method=cv2.RANSAC, prob=0.999, threshold=1.0)
                    
                    if E is None or E.shape != (3, 3):
                        print("Invalid essential matrix, repeating last trajectory")
                        x, y, z = trajectory.flatten()
                        writer.writerow([frame_num, x, y, z])
                        prev_frame_gray = curr_frame_gray.copy()
                        frame_num += 1
                        pbar.update(1)
                        continue
                    
                    _, R, t, _ = cv2.recoverPose(E, curr_points, prev_points, focal=1.0, pp=(0.0, 0.0))
                    
                    trajectory = trajectory + R @ t
                    poses.append(trajectory)
                    
                    x, y, z = trajectory.flatten()
                    writer.writerow([frame_num, x, y, z])
                    
                    prev_frame_gray = curr_frame_gray.copy()
                    prev_points = curr_points.reshape(-1, 1, 2)
                    
                    frame_num += 1
                except cv2.error as e:
                    print(f"Error processing frame: {e}, repeating last trajectory")
                    x, y, z = trajectory.flatten()
                    writer.writerow([frame_num, x, y, z])
                
                pbar.update(1)

def process_datasets(root_folder='EuRoC', mask_path=None, results_folder='camera'):
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)
    
    datasets = [os.path.join(root_folder, d) for d in os.listdir(root_folder) 
                if os.path.isdir(os.path.join(root_folder, d)) and not d.startswith('.')]
    
    for dataset in datasets:
        dataset_name = os.path.basename(dataset)
        csv_file_path = os.path.join(results_folder, f"{dataset_name}-results.csv")
        process_frames(dataset, mask_path, csv_file_path)

process_datasets(mask_path='mask.png')


Processing V1_02_medium:   0%|          | 0/1710 [00:00<?, ?it/s]

Not enough points to compute the essential matrix, re-detecting features
Error processing frame: OpenCV(4.9.0) /io/opencv/modules/calib3d/src/five-point.cpp:446: error: (-215:Assertion failed) npoints >= 0 && points2.checkVector(2) == npoints && points1.type() == points2.type() in function 'findEssentialMat'
, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Not enough points to compute the essential matrix, re-detecting features
Error processing frame: OpenCV(4.9.0) /io/opencv/modules/calib3d/src/five-point.cpp:446: error: (-215:Assertion failed) npoints >= 0 && points2.checkVector(2) == npoints && points1.type() == points2.type() in function 'findEssentialMat'
, repeating last trajectory
Not enough points to compute the essential matrix, re-detecting features
Error processing frame: OpenCV(4.9.0) /io/opencv/modules/calib3d/src/five-point.cpp:446: error:

Processing MH_05_difficult:   0%|          | 0/2273 [00:00<?, ?it/s]

Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid esse

Processing V1_01_easy:   0%|          | 0/2912 [00:00<?, ?it/s]

...

Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid esse

Processing MH_04_difficult:   0%|          | 0/2033 [00:00<?, ?it/s]

Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid esse

Processing MH_01_easy:   0%|          | 0/3682 [00:00<?, ?it/s]

Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid essential matrix, repeating last trajectory
Invalid esse